### Slicing — Advanced

This notebook goes deeper into Python slicing for built-in sequence types (`str`, `list`, `tuple`).

**You'll learn:**
- Slice anatomy and `slice` objects
- Default indices and negative steps (gotchas)
- Copying vs views (shallow copy semantics)
- Slice *assignment* for lists (insert/replace/delete/stride)
- `del` with slices
- Using `slice.indices()` to clamp ranges safely
- Idioms: reversing, sampling, chunking, windowing
- Common pitfalls (step = 0, mixing signs, etc.)

## 1) Slice objects vs literal syntax
Literal syntax `seq[start:stop:step]` is sugar for `seq[ slice(start, stop, step) ]`. You can build slices first, then reuse them.

- Defaults: `start=None`, `stop=None`, `step=None` (interpreted as 1).
- Step cannot be 0.
- Negative `step` walks right→left; bounds are interpreted accordingly.

In [1]:
s = 'Python rocks!'
sl = slice(7, None)     # same as s[7:]
sl2 = slice(None, 6)    # same as s[:6]
sl3 = slice(None, None, 2)  # same as s[::2]
s[sl], s[sl2], s[sl3]

('rocks!', 'Python', 'Pto ok!')

`slice.indices(len)` normalizes possibly negative/None bounds into concrete `(start, stop, step)` safely clamped to the sequence length. Great when writing utilities.

In [2]:
seq = [0,1,2,3,4,5]
sl = slice(-10, 999, 3)
sl.indices(len(seq))  # (start, stop, step) normalized

(0, 6, 3)

## 2) Defaults and negative steps — gotchas
- With a **negative** step, omitting `start` means "start from the last element".
- With a **negative** step, omitting `stop` means "go past the first element".
- If `start` and `stop` are inconsistent with the sign of `step`, you get an empty slice.

In [3]:
t = tuple('abcdef')
t[::-1], t[5:1:-1], t[1:5:-1]  # last is empty because 1..5 with -1 doesn't make sense

(('f', 'e', 'd', 'c', 'b', 'a'), ('f', 'e', 'd', 'c'), ())

**Zero step raises:**

In [4]:
try:
    _ = t[::0]
except ValueError as e:
    str(e)

## 3) Slices create **new containers**, but elements are **shared** (shallow copy)
Works the same for lists/tuples/strings: a new container is created, referencing the same element objects (if any are mutable, changes inside them are reflected).

In [5]:
l = [[0], [1], [2], [3]]
sub = l[1:3]
same_object = (sub[0] is l[1])  # True — shared inner list
sub[0].append(99)
l, sub, same_object

([[0], [1, 99], [2], [3]], [[1, 99], [2]], True)

**Full-slice is a common shallow copy idiom:** `l_copy = l[:]` (same as `list(l)`).

In [6]:
l = [1,2,3]
l_copy = l[:]
l is l_copy, l == l_copy

(False, True)

## 4) Slice **assignment** — lists only
Slice assignment lets you replace, insert, delete, or stride-replace multiple elements in one shot. Strings/tuples are immutable, so only lists support this.

In [7]:
# Replace a contiguous region (lengths may differ)
l = [0,1,2,3,4,5]
l[2:5] = [20, 30]          # shrink
l

# Insert via empty slice
l[2:2] = ['A','B']         # insertion at index 2
l

# Delete via empty RHS
l[1:4] = []
l

# Replace with step — lengths must match
l = [0,1,2,3,4,5,6]
l[::2] = [10,11,12,13]     # positions 0,2,4,6
l

[10, 1, 11, 3, 12, 5, 13]

**Mismatch raises `ValueError` when step is used:** RHS length must equal the number of positions addressed by the LHS slice when a non-`None` step is present.

In [8]:
try:
    l = [0,1,2,3,4,5]
    l[::2] = [99, 88]  # 3 targets vs 2 values
except ValueError as e:
    str(e)

### `del` with slices
`del seq[start:stop:step]` removes multiple items at once (lists only).

In [9]:
l = list(range(10))
del l[::3]     # delete indices 0,3,6,9
l  # [1,2,4,5,7,8]

[1, 2, 4, 5, 7, 8]

## 5) Useful idioms
- **Reverse:** `seq[::-1]`
- **Every k-th item (sampling):** `seq[::k]`
- **Tail (last n):** `seq[-n:]`
- **Head (first n):** `seq[:n]`
- **Drop head/tail:** `seq[n:]`, `seq[:-n]`
- **Rotate right by k:** `seq[-k:] + seq[:-k]` (works for strings/tuples/lists)
- **Chunking (non-overlapping):** use slicing in a loop or comprehension
- **Sliding windows (overlapping):** combine start/stop with window size

In [10]:
s = 'abcdefghij'
reverse = s[::-1]
sample3 = s[::3]
tail4 = s[-4:]
rotate2 = s[-2:] + s[:-2]
reverse, sample3, tail4, rotate2

('jihgfedcba', 'adgj', 'ghij', 'ijabcdefgh')

**Chunking** (size `k`, final chunk may be shorter):

In [11]:
def chunks(seq, k):
    return [seq[i:i+k] for i in range(0, len(seq), k)]

chunks('abcdefghij', 3), chunks(list(range(10)), 4)

(['abc', 'def', 'ghi', 'j'], [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]])

**Sliding windows** (size `k`, step `1`):

In [12]:
def sliding_windows(seq, k):
    if k <= 0:
        raise ValueError('k must be positive')
    return [seq[i:i+k] for i in range(0, len(seq)-k+1)]

sliding_windows('abcdef', 4), sliding_windows([1,2,3,4,5], 2)

(['abcd', 'bcde', 'cdef'], [[1, 2], [2, 3], [3, 4], [4, 5]])

## 6) Palindromes, normalization, and slicing
A quick palindrome check is `s == s[::-1]`. For robust checks on user input, normalize and casefold first (especially with Unicode).

In [13]:
import unicodedata as ud
def clean_text(x):
    x = ud.normalize('NFC', x).casefold()
    return ''.join(ch for ch in x if ch.isalnum())

def is_palindrome(text):
    c = clean_text(text)
    return c == c[::-1]

is_palindrome('Racecar'), is_palindrome('A man, a plan, a canal: Panama!'), is_palindrome('hello')

(True, True, False)

## 7) Performance notes
- Slicing runs in time proportional to the **result length** (needs to copy references/characters).
- Full-slice copies (`seq[:]`) are O(n); avoid in hot loops unless necessary.
- For immense strings, consider `io.StringIO`/`join` to build, rather than repeated concatenation. For lists, slice-assignment can be efficient for batch edits.

---
### Quick reference
- `seq[a:b]` → `a`..`b-1`
- `seq[a:b:c]` → step `c` (no `c=0`)
- Missing `a`/`b` default to start/end; with negative step they flip to end/start respectively
- New container (shallow copy); elements shared
- Lists: slice assignment & `del` slices mutate in bulk
- Use `slice.indices(len)` for safe bounds